<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>

# Reinforcement Learning from Human Feedback Using PPO

Estimated time needed: **30** minutes


Imagine you are an AI engineer who wants to train a "Happy LLM" and a "Pessimistic LLM" to train customer service agents. You have a reward function trained on the sentiment classifier from the IMDb dataset, and you will now use Reinforcement Learning (RL). RL is a subfield of machine learning where an agent learns to make decisions by performing actions in an environment to maximize a cumulative reward. The agent, in this case, will be the LLM, and the decisions will be about what text to output. Unlike supervised learning, which requires labeled input/output pairs, RL relies on the agent exploring the environment and learning from the feedback it receives in the form of rewards or penalties. This trial-and-error approach enables the agent to improve its decision-making strategy over time.

Proximal Policy Optimization (PPO) is one of the most effective and widely used RL algorithms. Introduced by OpenAI, PPO strikes a balance between simplicity and performance, making it a popular choice for training RL agents. PPO optimizes the policy directly and employs mechanisms to ensure the updates are not too drastic, thereby maintaining stability and reliability during training.

In this lab, you will be guided through the process of training an RL agent using the PPO algorithm with a focus on sentiment analysis. You will use the IMDb dataset, a large collection of movie reviews, to train your model. By the end of this lab, you will have a solid understanding of how to implement and train an RL agent using PPO, and you will be equipped with practical skills to apply RL techniques to other problems and datasets.
This lab is based on [a HF example code titled `Tune GPT2 to generate positive reviews`](https://github.com/huggingface/trl/blob/main/examples/notebooks/gpt2-sentiment.ipynb).



![rlhf](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/rlhf.png)


This image illustrates the process of **Reinforcement Learning with Human Feedback (RLHF)** applied to fine-tune a language model using a reward model and reinforcement learning techniques, such as Proximal Policy Optimization (PPO). Let me explain its key components step by step:

---

### 1. Prompts Dataset
- **Input**: A dataset of prompts $x$, such as "A dog is...".
- The prompts serve as the starting point for generating model outputs $y$, which are evaluated to improve the model's responses.

---

### 2. Initial Language Model
- **Base Model**:
  - The pretrained language model generates text $y$ for a given prompt $x$.
  - Example output: For the prompt "A dog is...", the base model might output "a furry mammal."

- **Purpose**:
  - This serves as the starting policy $\pi_{\text{base}}(y|x)$ before reinforcement learning is applied.

---

### 3. Tuned Language Model (RL Policy)
- **RL Fine-Tuning**:
  - The tuned language model $\pi_{\text{PPO}}(y|x)$ is updated using reinforcement learning. The parameters $\theta$ are adjusted through an iterative process to improve the model's alignment with human preferences.

- **Output**:
  - Example tuned response: "man's best friend," which aligns more closely with human expectations.

---

### 4. Reward Model (Preference Model)
- **Purpose**:
  - The reward model $r_\theta(y|x)$ evaluates the quality of the outputs $y$ for a given prompt $x$.
  - It is trained using human feedback (e.g., rankings, comparisons) to predict which outputs are preferred.

- **Input**: 
  - Text $x, y$ pairs.
  - Example: For the prompt "A dog is...", the reward model might score "man's best friend" higher than "a furry mammal."

---

### 5. Reinforcement Learning Update
- **PPO Update**:
  - The policy is updated via **Proximal Policy Optimization (PPO)**, which ensures stable learning by constraining the policy's updates.
  - The optimization objective maximizes the expected reward while penalizing large deviations from the base model $\pi_{\text{base}}(y|x)$.

---

### 6. KL Divergence Penalty
- **KL Regularization**:
  - The term $(-\lambda_{\text{KL}} D_{\text{KL}}(\pi_{\text{PPO}}(y|x) \,||\, \pi_{\text{base}}(y|x))$ penalizes the policy for straying too far from the initial language model's predictions.
  - **Why?** This regularization ensures that the fine-tuned model retains fluency and general knowledge from pretraining while still optimizing for the reward model.

---

### 7. Iterative Training Loop
- The process is iterative:
  1. **Generate Outputs**: The RL-tuned model generates outputs $y$ for prompts $x$.
  2. **Evaluate Outputs**: The reward model scores the outputs.
  3. **Update Model**: The PPO algorithm updates the model's parameters to maximize reward while staying close to the base model.

---

### Key Terms in the Formula
- **Reward $r_\theta(y|x)$**:
  - Guides the model to produce desirable outputs.
  
- **KL Divergence $D_{\text{KL}}$**:
  - Keeps the RL policy close to the pretrained base model.

- **PPO Update $\theta \gets \theta + \nabla_\theta J(\theta)$**:
  - Updates the model using gradients of the objective function.

---

### Summary
This diagram showcases the RLHF process, where:
1. A pretrained model is fine-tuned using PPO and a reward model.
2. The reward model, trained with human feedback, guides the optimization.
3. KL divergence regularization prevents the model from diverging too much from its base behavior, ensuring fluency and safety.




## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
            <li><a href="#Defining-helper-functions">Defining helper functions</a></li>
        </ol>
    </li>
    <li><a href="#Initializing-the-PPO-configuration,-model,-and-tokenizer">Initializing the PPO configuration, model, and tokenizer</a></li>
            <li><a href="#Dataset-and-dataset-tokenization">Dataset and dataset tokenization</a></li>
            <li><a href="#Collator-function">Collator function</a></li>
            <li><a href="#Initialize-PPOTrainer">Initialize PPOTrainer</a></li>
            <li><a href="#Reward-function">Reward function</a></li>
    <li>
        <a href="#Generating-responses-using-PPO">Generating responses using PPO</a>
        <ol>
            <li><a href="#Tokenizing-and-preparing-the-input-batch">Tokenizing and preparing the input batch</a></li>
            <li><a href="#Scoring-function">Scoring function</a></li>
            <li><a href="#Proximal-policy-optimization">Proximal policy optimization</a></li>
        </ol>
    </li>
    <li><a href="#Plotting-PPO-training-loss-and-mean">Plotting PPO training loss and mean</a></li>
    <li><a href="#Generating-and-analyzing-text-with-PPO-and-reference-models">Generating and analyzing text with PPO and reference models</a></li>
    <li>
        <a href="#Comparing-PPO-and-reference-models-on">Comparing PPO and reference models on</a>
        <ol>
        </ol>
    </li>
                <li><a href="#Running-the-PPO-model-with-negative-sentiment">Running the PPO model with negative sentiment</a></li>
            <li><a href="#Comparing-models-with-negative-sentiment">Comparing models with negative sentiment</a></li>
            <li><a href="#Exercise:-Comparing-PPO-models">Exercise: Comparing PPO models</a></li>
</ol>

## Objectives

After completing this lab you will be able to:

- Apply the basics of reinforcement learning and proximal policy optimization (PPO).
- Set up the environment and load the IMDb dataset for training.
- Define and configure the PPO agent and tokenizer.
- Implement the PPO training loop.
- Generate and evaluate text responses from the trained model.
- Compare the performance of two models on the dataset.
- Save and load the trained model for future use.

----

## Authors

[Joseph Santarcangelo](https://author.skills.network/instructors/joseph_santarcangelo) has a Ph.D. in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

[Ashutosh Sagar](https://www.linkedin.com/in/ashutoshsagar/) is completing his MS in CS from Dalhousie University. He has previous experience working with Natural Language Processing and as a Data Scientist.

## Contributors

[Hailey Quach](https://author.skills.network/instructors/hailey_quach) is a Data Scientist at IBM. She's completing her Bsc, Honors in Computer Science at Concordia University, Montreal.

## References

[RLHF: Reinforcement Learning from Human Feedback](https://huyenchip.com/2023/05/02/rlhf.html)

[Illustrating Reinforcement Learning from Human Feedback (RLHF)](https://huggingface.co/blog/rlhf)

[TEXT CLASSIFICATION WITH THE TORCHTEXT LIBRARY](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)

[Parameter-Efficient Transfer Learning for NLP](https://arxiv.org/pdf/1902.00751.pdf)

[Simple, Scalable Adaptation for Neural Machine Translation](https://arxiv.org/pdf/1909.08478)

```{|Date (YYYY-MM-DD)|Version|Changed By|Change Description||-|-|-|-||2024-06-27|0.1|Kang Wang|Create the lab|}
```

© Copyright IBM Corporation. All rights reserved.
